In [1]:
import pandas as pd
import numpy as np
from urllib import request
from bs4 import BeautifulSoup

from urllib.request import Request, urlopen

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.common.keys import Keys

import time

import urllib
import cv2

from tensorflow.keras.preprocessing import image

import matplotlib.pyplot as plt

import ssl

In [2]:
def find_kia_car():  # kia 홈페이지에서 현재 판매중인 차량 목록 가져오는 함수
    driver = webdriver.Chrome('chromedriver')
    url_list = ['ev', 'sedan', 'suv', 'commercial']  # kia 홈페이지에서 차량 분류한 방식
    
    global kia_list
    kia_list = []
    
    for i in range(len(url_list)):
        url = 'https://www.kia.com/kr/vehicles/' + url_list[i]  # 분류에 따라 url 접속
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # 차량 이름이 포함된 태그 select 후 변수로 저장
        info = soup.select('#contents > div > div > div > div.vehicis-cate__tabpanel-wrap.aem-Grid--12.aem-Grid--default--12')

        idx = 0
        while(True):
            try:
                # 차량 이름이 포함된 태그를 select한 변수에서 차량 이름만 가져와서 리스트에 저장
                kia_list.append(info[0].select('.vc-card__title')[idx].text.replace('\n',''))
                idx += 1
            except:
                break
    
    driver.close()
    
    return kia_list

In [3]:
find_kia_car()

['니로 EV',
 '니로 플러스',
 'EV6',
 'EV6 GT',
 '봉고Ⅲ EV',
 '봉고III 탑차/윙바디 EV',
 '봉고III 파워게이트 EV',
 '모닝',
 '레이',
 'K3',
 'K3 GT',
 'K5',
 'K8',
 '스팅어',
 'K9',
 '셀토스',
 '니로',
 '스포티지',
 '쏘렌토',
 '모하비',
 '카니발',
 '카니발 하이리무진',
 'K8택시',
 '봉고Ⅲ 트럭',
 '봉고III 탑차/윙바디/워크스루밴',
 '봉고III 덤프',
 '봉고III 파워게이트',
 '봉고III 와이드트럭/홈로리/활어수송차',
 '그랜버드']

In [4]:
def selenium_scroll(driver):  # 구글 검색 후 스크롤 내리는 함수
    SCROLL_PAUSE_SEC = 3
    i = 0
    while i<10:  # 스크롤 내린 후 더보기 버튼이 있다면 클릭
        i += 1    # 없으면 다시 스크롤 내리기 (10번 반복)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_SEC)
        try:
            btn =  driver.find_element(By.CSS_SELECTOR,'#islmp > div > div > div > div.gBPM8 > div.qvfT1 > div.YstHxe > input')
            btn.click()
        except:
            pass

In [5]:
def url_to_image(url, readFlag=cv2.IMREAD_COLOR):  # 이미지 url을 np.array로 바꿔주는 함수
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    gcontext = ssl.SSLContext()
    webpage = urlopen(req,context=gcontext,timeout=10)  # 정상적으로 열리지 않는 url이 있어서 timeout 옵션 부여
    image = np.asarray(bytearray(webpage.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    return image

In [6]:
kia = {}

In [6]:
def kia_crawling(carname):  # 크롤링하여 dictionary 형태로 저장하는 함수
    kia[carname] = []
    
    while(True):
        try:
            driver = webdriver.Chrome('chromedriver')  # 1. 구글 이미지 접속
            url = 'https://www.google.com/imghp'
            driver.get(url)
            break
        except:
            pass
        
    browser = driver.find_element(By.NAME,"q")  # 2. carname을 매개변수로 받아서 검색
    browser.send_keys(carname)
    browser.send_keys('\n')
    driver.implicitly_wait(3)
    
    selenium_scroll(driver)  # 3. 스크롤 끝까지 내리기
       
    time.sleep(3)
    
    html = driver.page_source  # 4. 스크롤 끝까지 내린 상태에서 페이지 소스 변수로 저장
    soup = BeautifulSoup(html, 'html.parser')
    
    imgs = driver.find_elements(By.CSS_SELECTOR, '#islrg > div.islrc > div a.wXeWr.islib.nfEiy')
       
    for i in range(len(imgs)):
        if i >= 500:  # 최대 500장의 이미지 저장
            break
            
        try:
            imgs[i].click()  # 5. 이미지 클릭
            time.sleep(1)
        except:
            break
        
        count = 0
        while(True):
            if count > 10:
                break
            count += 1
            try:
                # 6. 클릭한 이미지에서 url 정보 가져오기
                image = driver.find_element(By.CSS_SELECTOR, '#Sva75c > div > div > div.pxAole > div.tvh9oe.BIB1wf > c-wiz > div > div.OUZ5W > div.zjoqD > div.qdnLaf.isv-id > div > a')
                time.sleep(0.5)
                break
            except:
                pass
        
        count2 = 0
        while(True):
            if count2 > 5:
                break
            count2 += 1
            try:
                # 7. 가져온 url을 통해 이미지를 np.array로 저장 후 dictionary 안의 리스트에 저장
                image = image.find_element(By.TAG_NAME, 'img').get_attribute('src')
                image = url_to_image(image)
                kia[carname].append(image)
                break
            except:
                pass

    driver.close()

In [13]:
kia_crawling(kia_list[-1])

In [15]:
np.save('car project/kia.npy', kia)  # dictionary 저장

In [7]:
kia = np.load(('car project/kia.npy'),allow_pickle='TRUE').item()  # 저장한 .npy 파일 불러오기

In [20]:
kia.keys()  # 정상적으로 불러왔는지 확인

dict_keys(['니로 EV', '니로 플러스', 'EV6', 'EV6 GT', '봉고Ⅲ EV', '봉고III 탑차/윙바디 EV', '봉고III 파워게이트 EV', '모닝', 'K3', 'K3 GT', 'K5', 'K8', '기아 레이', '스팅어', 'K9', '셀토스', '니로', '스포티지', '쏘렌토', '모하비', '카니발', '카니발 하이리무진', 'K8택시', '봉고Ⅲ 트럭', '봉고III 탑차/윙바디/워크스루밴', '봉고III 덤프', '봉고III 파워게이트', '봉고III 와이드트럭/홈로리/활어수송차', '그랜버드'])